# Resonant Weakly Forced Rayleigh Oscillator #

Support material for the book __Perturbation Methods from a Backward Error Point of View__ by Robert M. Corless and Nicolas Fillion, in preparation.

Copyright (c) 2024 Robert M. Corless

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

$\newcommand{\e}{\varepsilon}$

This notebook explores the __weakly__ forced Rayleigh oscillator
\begin{equation}
\ddot{y} - \e \dot{y}\left( 1 - \frac{4}{3}\dot{y}^2 \right) + y = 2\e F\cos(\Omega t + \Phi)
\end{equation}
for $\Omega \approx 1$ by perturbation expansion up to and including the $O(\e^N)$ terms, so it has a residual $O(\e^{N+1})$.  A strong forcing near resonance would get an $O(1/\e)$ response, so we examine this case.

Define $N$ in the next cell.

In [1]:
N := 2;

kilobytes used=1805, alloc=5424, time=0.30

$$2$$

## Weakly forced resonant case ##

In [2]:
macro( e=varepsilon ):
Omega := 1 + sigma*e/2: # conventional
WeaklyNonlineart := diff(y(t), t, t) 
  - e*diff(y(t), t)*(1 - 4/3*diff(y(t), t)^2) + y(t) 
  - e*F*exp(I*(tau+Phi)) - e*F*exp(-I*(tau+Phi)) :
tmp := PDETools:-dchange( t=tau/Omega, WeaklyNonlineart, [tau]):
WeaklyNonlinear := eval(tmp, y(tau,sigma,e)=y(tau));

$$\left(\frac{d^{2}}{d \tau^{2}}y \left(\tau \right)\right) \left(1+\frac{\sigma  \varepsilon}{2}\right)^{2}-\varepsilon  \left(\frac{d}{d \tau}y \left(\tau \right)\right) \left(1+\frac{\sigma  \varepsilon}{2}\right) \left(1-\frac{4 \left(\frac{d}{d \tau}y \left(\tau \right)\right)^{2} \left(1+\frac{\sigma  \varepsilon}{2}\right)^{2}}{3}\right)+y \left(\tau \right)-\varepsilon  F \,{\mathrm e}^{\mathrm{I} \left(\tau +\Phi \right)}-\varepsilon  F \,{\mathrm e}^{\mathrm{-I} \left(\tau +\Phi \right)}$$

In [3]:
LinearODE := diff(y(tau),tau,tau) + y(tau);

$$\frac{d^{2}}{d \tau^{2}}y \left(\tau \right)+y \left(\tau \right)$$

In [4]:
sol := dsolve( eval( WeaklyNonlinear, e=0), y(tau) );

$$y \left(\tau \right) = c_{1} \sin \left(\tau \right)+c_{2} \cos \left(\tau \right)$$

That solution has sines and cosines.  We will work over the complex exponentials first.  Use the variable zs to keep the solution in.

In [5]:
ys := Array(0 .. N): # Placeholder for secular solution terms
rs := Array(0 .. N): # Placeholder for residuals
ys[0] :=  A*exp(I*phi)*exp(I*tau) + A*exp(-I*phi)*exp(-I*tau) ;
zs := ys[0];

         ys[0] := A exp(phi I) exp(tau I) + A exp(-I phi) exp(-I tau)

$$A \,{\mathrm e}^{\phi  \mathrm{I}} {\mathrm e}^{\tau  \mathrm{I}}+A \,{\mathrm e}^{\mathrm{-I} \phi} {\mathrm e}^{\mathrm{-I} \tau}$$

We will continue to work with sines and cosines, but to get the secular equation we will convert to exponentials.

In [6]:
# SHOlveTerm --- solve a simple harmonic oscillator, harmonically forced
#
# Copyright 2024 Robert M. Corless 
#
# dsolve can be inefficient at solving y'' + omega^2*y = sum of simple terms
# (probably getting bogged down in type checking or simplification
#  or its own heuristics, or perhaps it's just its generality).
# This routine attempts to do this simpler task more efficiently,
# by giving up on generality.
#
# Input: expr  : right-hand side: polynomial in sine/cos/exp only
#                with coefficients polynomials in var
#        var   : variable (usually t)
#        omega : natural frequency
#
# Processing: map over a sum of terms
#             match the terms to poly(var)*exp( I*(freq*var + phase) )
#                            or  poly(var)*sin( freq*var + phase )
#                            or  poly(var)*cos( freq*var + phase )
#
#          solve the problem y'' + omega^2*y = sum poly(var)* fn( freq*var+phase )
#
# Output: a particular solution to y'' + omega^2 = ...
#         where the rhs is a sum of polynomials in var times sin/cos/exp.
#
# Comments: dsolve would solve this, but for large expressions might take
# a seemingly unreasonable length of time to do so, and may return 
# unevaluated integrals (because it thinks the answers are too complicated).
# However, this simple routine will not succeed for unexpected inputs.
# Take your pick.

SHOlveTerm := proc( expr, var, omega )
	local A, c, fn, freq, la, m, phase, sol, term, zerp;
	description "Solve y''+ omega^2*y=expr";
    if has( expr, sin ) or has( expr, cos ) then
      # convert,exp will make cos(A) --> (exp(I*A)+exp(-I*A))/2, etc
      # expand will expand (A*exp(B) + C*exp(D))*(E*exp(F)+G*exp(H))
      # to A*E*exp(B)*exp(F) + A*G*exp(B)*exp(H) + ... 
      # combine,exp will make exp(B)*exp(F)-->exp(B+F)
      term := combine( expand( convert(expr,exp ) ), exp );
    else
      term := expr;
    end if; 
	if type( term, `+` ) then
      # Apply this procedure to each term in a sum  
	  return map( procname, term, var, omega );
	else 
      # Look for A*exp(I*(freq*var+phase))
      term := combine( term, exp );
      A := eval(term, exp=1);
      fn := normal( term/A );
      # Each term *should* be A*exp(B).  Should be.
      if normal( expr-A*fn ) <> 0 then
        error "Unexpected term type", term;
      end if;
      # Identify frequency and phase
      if match( fn=exp(I*(freq*var+phase)), var, 'la' ) then
        if eval(freq,la)^2 = omega^2 then
          # resonant case
          m := degree(A, var) + 1; # A must be polynomial
        else
          m := degree(A, var);
        end if;
        P := add( c[k]*var^k, k=0..m );
        # y = P(var)*exp(I*(freq*var+phase))
        # for P(var) a polynomial to be found.
        zerp := eval( omega^2*P + diff(P,var,var) + 2*I*freq*diff(P,var) - freq^2*P - A, la);
        sol := solve( {seq( coeff(zerp,var,k), k=0..m)}, {seq(c[k],k=0..m)} );
        # triangular system, guaranteed to have a solution
        # Throw away the redundant terms c[0]*exp(I*(freq*var+phase))
        return eval( eval( eval(P,sol), c[0]=0)*exp(I*(freq*var+phase)), la );
      else
        error "Unexpected term type, after match", term, A, fn;
      end if;
    end if;
end proc:

In [7]:
loop_start := time():
times := Array(1 .. N):
for k to N do
    residuals[k - 1] := combine(expand(
                          coeff(eval(WeaklyNonlinear, y(tau) = zs), e, k)),
                          exp);
    newans := SHOlveTerm( residuals[k-1], tau, 1 );
    ys[k] := -newans;
    zs := zs + ys[k]*e^k;
    times[k] := time();
end do:

kilobytes used=6066, alloc=9818, time=0.69

In [8]:
resonant := 0; # This is not particularly robust.
ze := expand( combine( expand(zs), exp ) ):
trms := [op(ze)];
detected := Array(1..numelems(trms), j->false):
for k to numelems( trms ) do
  trm := combine( trms[k], exp );
  conpart := eval(trm,exp=1); # trm = A*exp(B) we hope
  expart := normal( trm/conpart );
  if match( combine(expart,exp)=exp(I*tau + phase), tau, 'la') then
    resonant := resonant + conpart*eval(exp(phase),la);
    detected[k] := true;
  end if;
end do:

                                                       exp(Phi I) exp(tau I)

In [9]:
yA := resonant/(A*exp(I*phi)):

In [10]:
assumption_list := R>0, F>0, e>0, phi::real, Phi::real, tau>0, A>0, sigma::real;

$$0<R,0<F,0<\varepsilon,\phi ::\mathit{real},\Phi ::\mathit{real},0<\tau,0<A,\sigma ::\mathit{real}$$

In [11]:
rhsR := simplify( Re( convert(series( diff(yA,tau)/yA,e,N+1),polynom) ) ) 
        assuming assumption_list:

kilobytes used=12628, alloc=14426, time=1.16

We know $A=R + O(\e)$ but we need to know more, now.

In [12]:
yA := collect(yA, e, m -> Re(m) + Im(m)*I) assuming assumption_list;
yA := combine(yA, trig);
#Now we will need to invert R = A*yA and the best way is by a real equation.  R is real and A > 0 by our choice of phi.
rhosq := series(evalc(Re(yA)^2 + Im(yA)^2), e, N + 1);
rhosq := map(simplify, rhosq);
rhosq := convert(rhosq, polynom);
#Our equation to solve for A in terms of R and e is
freqn := -A^2*rhosq + R^2;
#We will solve this perturbatively for A = g(R,e).  We need the derivative, for our regular perturbation method.
eval(diff(freqn, R), e = 0);
#So our inverse operator is 1/2R.  Now check that our initial approximation A=R is accurate enough:
series(leadterm(eval(freqn, A = R)), e);
#So A = R is a solution to Order epsilon
#Let's use the array "Eh" to represent our terms in the A expansion.
Eh := Array(0 .. N);
#We have to put the residuals somewhere:
residEh := Array(0 .. N);
#Our initial approximation:
Eh[0] := R;
#Our running solution is initially the same as A[0] (I mean, Eh[0]).
Ehz := Eh[0];
#Regular perturbation solution of the freqn
for k to N do
    residEh[k - 1] := coeff(map(simplify, series(eval(freqn, A = Ehz), e, k + 2)), e, k);
    Eh[k] := residEh[k - 1]*e^k/(2*R);
    Ehz := Ehz + Eh[k];
end do;
#Check the final residual
residEh[N] :=  
  collect(convert( map( simplify, series( eval(freqn, A=Ehz), e, N+2 ) ),polynom),
          e,LargeExpressions:-Veil[K]);


                     residEh[2] := - 1/8 K[1] varepsilon

In [13]:
collect( LargeExpressions:-Unveil[K]( K[4] ), [sin,cos]);

$$K_{4}$$

It's very puzzling that the above doesn't detect that the coefficients are zero, while at the same time the identical command below _does_ detect it.

In [14]:
map( simplify, series( eval(freqn, A=Ehz), e, N+2 ) );

$$-\frac{3 \left(\frac{F^{2} \sin \left(-\phi +\Phi \right)^{2} \sigma}{4}-5 F \left(F \left(R^{2}+\frac{\tau  \sigma}{60}-\frac{1}{60}\right) \cos \left(-\phi +\Phi \right)+\frac{8 R \left(R^{4} \tau +\frac{\left(-\tau +\frac{5 \sigma}{2}\right) R^{2}}{3}+\frac{\tau}{48}-\frac{5 \sigma}{48}\right)}{5}\right) \sin \left(-\phi +\Phi \right)+40 R F \left(\frac{1}{240}+R^{4}+\frac{\left(-1+\frac{\tau  \sigma}{6}\right) R^{2}}{5}-\frac{\tau  \sigma}{240}\right) \cos \left(-\phi +\Phi \right)+16 R^{8} \tau +\frac{28 \left(-\tau +2 \sigma \right) R^{6}}{3}+\frac{5 \left(\tau -4 \sigma \right) R^{4}}{3}+\left(F^{2} \tau -\frac{1}{12} \tau +\frac{1}{2} \sigma \right) R^{2}-\frac{F^{2} \tau}{12}\right) \tau^{2}}{2} \varepsilon^{3}+\mathrm{O}\left(\varepsilon^{4}\right)$$

In [15]:
rhsR := convert( series( eval(rhsR,A=Ehz), e, N+1), polynom ):

In [16]:
rhsR := collect( rhsR, [e,F,R], distributed, factor ): 

In [17]:
slowRderiv := R*convert( rhsR, polynom ):

In [18]:
`diff/R` := codegen[makeproc](eval(slowRderiv*'diff'('expr', 'var'), 
  [R = R(expr), phi = theta(expr),tau=expr]), parameters = [expr, var]);

proc (expr, var) R(expr)*(1/4*expr*(sin(-theta(expr)+Phi)^2+cos(-2*theta(expr)+2*Phi))/R(expr)^2*varepsilon^2*F^2+5/2*cos(-theta(expr)+Phi)*varepsilon^2*F*R(expr)+(-3/8*sigma*sin(-theta(expr)+Phi)+1/4*sigma*expr*cos(-theta(expr)+Phi)-1/8*cos(-theta(expr)+Phi))*varepsilon^2*F/R(expr)+sigma*varepsilon^2*R(expr)^2-1/4*sigma*varepsilon^2+1/2*sin(-theta(expr)+Phi)/R(expr)*varepsilon*F-2*varepsilon*R(expr)^2+1/2*varepsilon)*diff(expr,var) end proc

In [19]:
diff(R(tau),tau);

$$R \left(\tau \right) \left(\frac{\tau  \left(\sin \left(-\theta \left(\tau \right)+\Phi \right)^{2}+\cos \left(-2 \theta \left(\tau \right)+2 \Phi \right)\right) \varepsilon^{2} F^{2}}{4 R \left(\tau \right)^{2}}+\frac{5 \cos \left(-\theta \left(\tau \right)+\Phi \right) \varepsilon^{2} F R \left(\tau \right)}{2}+\frac{\left(-\frac{3 \sigma  \sin \left(-\theta \left(\tau \right)+\Phi \right)}{8}+\frac{\sigma  \tau  \cos \left(-\theta \left(\tau \right)+\Phi \right)}{4}-\frac{\cos \left(-\theta \left(\tau \right)+\Phi \right)}{8}\right) \varepsilon^{2} F}{R \left(\tau \right)}+\sigma  \,\varepsilon^{2} R \left(\tau \right)^{2}-\frac{\sigma  \,\varepsilon^{2}}{4}+\frac{\sin \left(-\theta \left(\tau \right)+\Phi \right) \varepsilon  F}{2 R \left(\tau \right)}-2 \varepsilon  R \left(\tau \right)^{2}+\frac{\varepsilon}{2}\right)$$

In [20]:
rhstheta := Im( 
             convert( 
               simplify( series( eval( diff(yA,tau)/yA, A=Ehz ),e,N+1))
               ,polynom)
            ) assuming assumption_list;

kilobytes used=26216, alloc=18522, time=2.22

$$\frac{\varepsilon  \left(-F \cos \left(-\phi +\Phi \right)-R \sigma \right)}{2 R}+\frac{\varepsilon^{2} \left(8 R^{6}+2 F^{2} \sin \left(-\phi +\Phi \right) \cos \left(-\phi +\Phi \right) \tau -4 R^{3} F \sin \left(-\phi +\Phi \right)+2 \tau  F R \sin \left(-\phi +\Phi \right) \sigma +3 F \cos \left(-\phi +\Phi \right) \sigma  R +2 R^{2} \sigma^{2}-F \sin \left(-\phi +\Phi \right) R -R^{2}\right)}{8 R^{2}}$$

In [21]:
rhstheta := combine(rhstheta,trig);

$$\frac{\varepsilon  \left(8 R^{6} \varepsilon -4 F \sin \left(-\phi +\Phi \right) R^{3} \varepsilon +2 F \sin \left(-\phi +\Phi \right) R \sigma  \tau  \varepsilon +F^{2} \sin \left(-2 \phi +2 \Phi \right) \tau  \varepsilon +3 F \cos \left(-\phi +\Phi \right) R \sigma  \varepsilon +2 R^{2} \sigma^{2} \varepsilon -F \sin \left(-\phi +\Phi \right) R \varepsilon -4 F R \cos \left(-\phi +\Phi \right)-4 R^{2} \sigma -\varepsilon  \,R^{2}\right)}{8 R^{2}}$$

In [22]:
`diff/theta` := codegen[makeproc](eval(rhstheta*'diff'('expr', 'var'), 
  [R = R(expr), phi = theta(expr), tau=expr]), parameters = [expr, var]);

proc (expr, var) 1/8*varepsilon*(8*R(expr)^6*varepsilon-4*F*sin(-theta(expr)+Phi)*R(expr)^3*varepsilon+2*F*sin(-theta(expr)+Phi)*R(expr)*sigma*expr*varepsilon+F^2*sin(-2*theta(expr)+2*Phi)*expr*varepsilon+3*F*cos(-theta(expr)+Phi)*R(expr)*sigma*varepsilon+2*R(expr)^2*sigma^2*varepsilon-F*sin(-theta(expr)+Phi)*R(expr)*varepsilon-4*F*R(expr)*cos(-theta(expr)+Phi)-4*R(expr)^2*sigma-varepsilon*R(expr)^2)/R(expr)^2*diff(expr,var) end proc

In [23]:
diff(theta(tau),tau);

$$\frac{\varepsilon  \left(8 R \left(\tau \right)^{6} \varepsilon -4 F \sin \left(-\theta \left(\tau \right)+\Phi \right) R \left(\tau \right)^{3} \varepsilon +2 F \sin \left(-\theta \left(\tau \right)+\Phi \right) R \left(\tau \right) \sigma  \tau  \varepsilon +F^{2} \sin \left(-2 \theta \left(\tau \right)+2 \Phi \right) \tau  \varepsilon +3 F \cos \left(-\theta \left(\tau \right)+\Phi \right) R \left(\tau \right) \sigma  \varepsilon +2 R \left(\tau \right)^{2} \sigma^{2} \varepsilon -F \sin \left(-\theta \left(\tau \right)+\Phi \right) R \left(\tau \right) \varepsilon -4 F R \left(\tau \right) \cos \left(-\theta \left(\tau \right)+\Phi \right)-4 R \left(\tau \right)^{2} \sigma -\varepsilon  R \left(\tau \right)^{2}\right)}{8 R \left(\tau \right)^{2}}$$

Now we re-do the computation (we should have checked the residual up there to see if it was $O(\e^2)$, but we will do it once we have recomputed).

In [24]:
yr := Array(0..N);
rr := Array(0..N);
# Start with the right initial approximation
yr[0] := collect( combine( convert( eval( ys[0], 
  [A=R(tau), phi=theta(tau)]), trig ), trig ), [R(tau),F], normal ) 
  assuming assumption_list;

                        rr := Array(0 .. 2, [0, 0, 0])

$$2 R \left(\tau \right) \cos \left(\theta \left(\tau \right)+\tau \right)$$

To solve the first-order correction, we have to hide the dependence of R and theta on t from dsolve.  Because their derivatives are $O(\e)$, they are constant for the purpose of perturbative correction.

In [25]:
loop_start := time():
timesr := Array(1 .. N):
zr := yr[0];
for k to N do
    res := series( eval(WeaklyNonlinear, y(tau) = zr), e, k+1 );
    rr[k - 1] := combine(expand(
                          coeff( res, e, k)),
                          exp);
    tmp := eval(rr[k-1], [R(tau)=RR,theta(tau)=THETA]);
    newans := SHOlveTerm( tmp, tau, 1 );
    newans := eval( newans, [RR=R(tau), THETA=theta(tau)]);
    yr[k] := -newans;
    zr := zr + yr[k]*e^k;
    timesr[k] := time();
end do:

kilobytes used=36701, alloc=18522, time=2.97

In [26]:
zr;

$$2 R \left(\tau \right) \cos \left(\theta \left(\tau \right)+\tau \right)+\left(-\frac{\mathrm{I} R \left(\tau \right)^{3} {\mathrm e}^{\mathrm{I} \left(3 \theta \left(\tau \right)+3 \tau \right)}}{6}+\frac{\mathrm{I} R \left(\tau \right)^{3} {\mathrm e}^{\mathrm{I} \left(-3 \theta \left(\tau \right)-3 \tau \right)}}{6}\right) \varepsilon +\left(-\frac{R \left(\tau \right)^{5} {\mathrm e}^{\mathrm{I} \left(5 \theta \left(\tau \right)+5 \tau \right)}}{12}-\frac{R \left(\tau \right)^{3} {\mathrm e}^{\mathrm{I} \left(3 \theta \left(\tau \right)+3 \tau \right)}}{8}-\frac{R \left(\tau \right)^{3} {\mathrm e}^{\mathrm{I} \left(-3 \theta \left(\tau \right)-3 \tau \right)}}{8}-\frac{R \left(\tau \right)^{5} {\mathrm e}^{\mathrm{I} \left(-5 \theta \left(\tau \right)-5 \tau \right)}}{12}+\frac{3 R \left(\tau \right)^{5} {\mathrm e}^{\mathrm{I} \left(3 \theta \left(\tau \right)+3 \tau \right)}}{4}-\frac{\mathrm{I} R \left(\tau \right)^{2} F \,{\mathrm e}^{\mathrm{I} \left(-3 \tau -\Phi -2 \theta \left(\tau \right)\right)}}{16}-\frac{\mathrm{I} F^{2} \tau  \,{\mathrm e}^{\mathrm{I} \left(-\theta \left(\tau \right)-\tau \right)}}{16 R \left(\tau \right)}+\frac{\mathrm{I} R \left(\tau \right)^{2} F \,{\mathrm e}^{\mathrm{I} \left(3 \tau +\Phi +2 \theta \left(\tau \right)\right)}}{16}+\frac{\mathrm{I} F^{2} \tau  \,{\mathrm e}^{\mathrm{I} \left(\theta \left(\tau \right)+\tau \right)}}{16 R \left(\tau \right)}+\frac{\mathrm{I} \sigma  F \tau  \,{\mathrm e}^{\mathrm{I} \left(\tau +\Phi \right)}}{8}-\left(\frac{\sigma  F \,\tau^{2}}{8}+\frac{\mathrm{I} \sigma  F \tau}{8}\right) {\mathrm e}^{\mathrm{I} \left(\tau +\Phi \right)}-\left(\frac{F^{2} \tau^{2}}{16 R \left(\tau \right)}+\frac{\mathrm{I} F^{2} \tau}{16 R \left(\tau \right)}\right) {\mathrm e}^{\mathrm{I} \left(\tau +2 \Phi -\theta \left(\tau \right)\right)}-\left(\frac{F^{2} \tau^{2}}{16 R \left(\tau \right)}-\frac{\mathrm{I} F^{2} \tau}{16 R \left(\tau \right)}\right) {\mathrm e}^{\mathrm{I} \left(-\tau -2 \Phi +\theta \left(\tau \right)\right)}-\left(\frac{F^{2} \tau^{2}}{16 R \left(\tau \right)}-\frac{\mathrm{I} F^{2} \tau}{16 R \left(\tau \right)}\right) {\mathrm e}^{\mathrm{I} \left(-\theta \left(\tau \right)-\tau \right)}+\frac{3 R \left(\tau \right)^{5} {\mathrm e}^{\mathrm{I} \left(-3 \theta \left(\tau \right)-3 \tau \right)}}{4}-\left(\frac{\sigma  F \,\tau^{2}}{8}-\frac{\mathrm{I} \sigma  F \tau}{8}\right) {\mathrm e}^{\mathrm{I} \left(-\Phi -\tau \right)}-\left(\frac{F^{2} \tau^{2}}{16 R \left(\tau \right)}+\frac{\mathrm{I} F^{2} \tau}{16 R \left(\tau \right)}\right) {\mathrm e}^{\mathrm{I} \left(\theta \left(\tau \right)+\tau \right)}-\frac{\mathrm{I} \sigma  F \tau  \,{\mathrm e}^{\mathrm{I} \left(-\Phi -\tau \right)}}{8}+\frac{\mathrm{I} F^{2} \tau  \,{\mathrm e}^{\mathrm{I} \left(\tau +2 \Phi -\theta \left(\tau \right)\right)}}{16 R \left(\tau \right)}-\frac{\mathrm{I} F^{2} \tau  \,{\mathrm e}^{\mathrm{I} \left(-\tau -2 \Phi +\theta \left(\tau \right)\right)}}{16 R \left(\tau \right)}\right) \varepsilon^{2}$$

Some combination of simplification commands cleans up the answer to be intelligible.

In [27]:
ztrig := collect( combine(convert(zr,trig),trig), 
  [sin,cos], m->collect(m,[R(tau),F], factor));

$$\frac{R \left(\tau \right)^{3} \sin \left(3 \theta \left(\tau \right)+3 \tau \right) \varepsilon}{3}-\frac{F R \left(\tau \right)^{2} \varepsilon^{2} \sin \left(3 \tau +\Phi +2 \theta \left(\tau \right)\right)}{8}-\frac{F \sigma  \,\tau^{2} \varepsilon^{2} \cos \left(\tau +\Phi \right)}{4}+\left(2 R \left(\tau \right)-\frac{F^{2} \tau^{2} \varepsilon^{2}}{8 R \left(\tau \right)}\right) \cos \left(\theta \left(\tau \right)+\tau \right)+\left(\frac{3 \varepsilon^{2} R \left(\tau \right)^{5}}{2}-\frac{\varepsilon^{2} R \left(\tau \right)^{3}}{4}\right) \cos \left(3 \theta \left(\tau \right)+3 \tau \right)-\frac{\varepsilon^{2} R \left(\tau \right)^{5} \cos \left(5 \theta \left(\tau \right)+5 \tau \right)}{6}-\frac{F^{2} \cos \left(\tau +2 \Phi -\theta \left(\tau \right)\right) \tau^{2} \varepsilon^{2}}{8 R \left(\tau \right)}$$

The fact that _this_ final residual is $O(\e^(N+1))$ proves that we have found a solution to the right order.

In [28]:
fullresidual := eval(WeaklyNonlinear,y(tau)=ztrig):
map(LargeExpressions:-Veil[K], map(simplify, series(fullresidual, e, N+2)));

kilobytes used=44675, alloc=18522, time=3.47

$$-\frac{K_{2}}{24} \varepsilon^{3}+K_{3} \varepsilon^{4}$$

In [29]:
collect( combine(
   coeff( series( fullresidual, e, N+2), e, N+1 ),
                 trig ),
   [R(tau),F,sin,cos], factor );

kilobytes used=51312, alloc=18522, time=3.91

$$\left(-\frac{16 \sin \left(7 \theta \left(\tau \right)+7 \tau \right)}{3}+\frac{74 \sin \left(3 \theta \left(\tau \right)+3 \tau \right)}{3}+34 \sin \left(5 \theta \left(\tau \right)+5 \tau \right)-26 \sin \left(\theta \left(\tau \right)+\tau \right)\right) R \left(\tau \right)^{7}+\left(-2 \cos \left(\theta \left(\tau \right)+\tau \right) \sigma -7 \sin \left(3 \theta \left(\tau \right)+3 \tau \right)+11 \sin \left(\theta \left(\tau \right)+\tau \right)-\frac{17 \sin \left(5 \theta \left(\tau \right)+5 \tau \right)}{3}\right) R \left(\tau \right)^{5}+\left(-\frac{23 \cos \left(\tau +\Phi \right)}{2}+6 \cos \left(-2 \theta \left(\tau \right)+\Phi -\tau \right)-\frac{25 \cos \left(-4 \theta \left(\tau \right)-3 \tau +\Phi \right)}{2}+\frac{10 \cos \left(4 \theta \left(\tau \right)+5 \tau +\Phi \right)}{3}-2 \cos \left(3 \tau +\Phi +2 \theta \left(\tau \right)\right)\right) F R \left(\tau \right)^{4}+\left(-2 \sin \left(\theta \left(\tau \right)+\tau \right) \sigma^{2}-\frac{\sin \left(3 \theta \left(\tau \right)+3 \tau \right)}{2}\right) R \left(\tau \right)^{3}+\left(2 \sigma  \left(\tau^{2}-2\right) \sin \left(\tau +\Phi \right)-\frac{\sigma  \left(8 \tau^{2}-9\right) \sin \left(3 \tau +\Phi +2 \theta \left(\tau \right)\right)}{8}-\sigma  \left(\tau^{2}-3\right) \sin \left(-2 \theta \left(\tau \right)+\Phi -\tau \right)-2 \sigma  \tau  \cos \left(\tau +\Phi \right)+\left(\frac{3 \tau  \sigma}{2}+\frac{7}{8}\right) \cos \left(3 \tau +\Phi +2 \theta \left(\tau \right)\right)+\left(-\tau  \sigma -1\right) \cos \left(-2 \theta \left(\tau \right)+\Phi -\tau \right)\right) F R \left(\tau \right)^{2}+\left(\left(2 \left(\tau -1\right) \left(\tau +1\right) \sin \left(\theta \left(\tau \right)+\tau \right)-\frac{\left(2 \tau -1\right) \left(2 \tau +1\right) \sin \left(3 \theta \left(\tau \right)+3 \tau \right)}{8}+\left(\frac{3 \tau^{2}}{2}-1\right) \sin \left(\tau +2 \Phi -\theta \left(\tau \right)\right)-\frac{\tau^{2} \sin \left(-3 \theta \left(\tau \right)+2 \Phi -\tau \right)}{2}+\left(-\frac{\tau^{2}}{2}+\frac{7}{8}\right) \sin \left(\theta \left(\tau \right)+3 \tau +2 \Phi \right)-\frac{3 \cos \left(\tau +2 \Phi -\theta \left(\tau \right)\right) \tau}{2}-2 \cos \left(\theta \left(\tau \right)+\tau \right) \tau +\frac{3 \cos \left(\theta \left(\tau \right)+3 \tau +2 \Phi \right) \tau}{4}+\frac{3 \cos \left(3 \theta \left(\tau \right)+3 \tau \right) \tau}{4}-\frac{\tau  \cos \left(-3 \theta \left(\tau \right)+2 \Phi -\tau \right)}{2}\right) F^{2}+\frac{\sin \left(\theta \left(\tau \right)+\tau \right) \sigma^{2}}{2}-\frac{\sigma  \left(2 \sigma^{2}-1\right) \cos \left(\theta \left(\tau \right)+\tau \right)}{4}\right) R \left(\tau \right)+\left(\frac{\sigma  \left(\tau  \sigma -\tau^{2}+2\right) \sin \left(\tau +\Phi \right)}{4}+\frac{\sigma  \left(2 \tau^{2} \sigma -7 \sigma +2 \tau \right) \cos \left(\tau +\Phi \right)}{8}\right) F +\frac{\left(-\frac{\sin \left(\theta \left(\tau \right)+\tau \right) \tau^{2}}{4}+\frac{\tau  \left(-\tau +\sigma \right) \sin \left(\tau +2 \Phi -\theta \left(\tau \right)\right)}{4}+\frac{\cos \left(\theta \left(\tau \right)+\tau \right) \tau}{4}+\frac{\tau  \left(\tau  \sigma +1\right) \cos \left(\tau +2 \Phi -\theta \left(\tau \right)\right)}{4}\right) F^{2}}{R \left(\tau \right)}+\frac{\left(-\frac{\tau^{2} \cos \left(-2 \theta \left(\tau \right)+\Phi -\tau \right)}{8}+\frac{\cos \left(-2 \theta \left(\tau \right)+3 \Phi +\tau \right) \tau^{2}}{8}+\frac{\tau  \sin \left(-2 \theta \left(\tau \right)+\Phi -\tau \right)}{8}+\frac{\tau  \sin \left(-2 \theta \left(\tau \right)+3 \Phi +\tau \right)}{8}\right) F^{3}}{R \left(\tau \right)^{2}}$$

Bad: secular terms visible.

In [30]:
series( diff(R(tau),tau), e, N+1 );

$$R \left(\tau \right) \left(\frac{\sin \left(-\theta \left(\tau \right)+\Phi \right) F}{2 R \left(\tau \right)}-2 R \left(\tau \right)^{2}+\frac{1}{2}\right) \varepsilon +R \left(\tau \right) \left(\frac{\tau  \left(\sin \left(-\theta \left(\tau \right)+\Phi \right)^{2}+\cos \left(-2 \theta \left(\tau \right)+2 \Phi \right)\right) F^{2}}{4 R \left(\tau \right)^{2}}+\frac{5 F R \left(\tau \right) \cos \left(-\theta \left(\tau \right)+\Phi \right)}{2}+\frac{\left(-\frac{3 \sigma  \sin \left(-\theta \left(\tau \right)+\Phi \right)}{8}+\frac{\sigma  \tau  \cos \left(-\theta \left(\tau \right)+\Phi \right)}{4}-\frac{\cos \left(-\theta \left(\tau \right)+\Phi \right)}{8}\right) F}{R \left(\tau \right)}+R \left(\tau \right)^{2} \sigma -\frac{\sigma}{4}\right) \varepsilon^{2}$$

In [33]:
RT := convert( eval( series( R(tau), tau, N+1 ), [theta(0)=phi,R(0)=A]), polynom):
thetat := convert( eval( series( theta(tau), tau, N+1 ), [theta(0)=phi,R(0)=A]), polynom):
combine( convert( coeff(zs,e,0), trig ),trig):
maybe := series( eval(ztrig, [R(tau)=RT,theta(tau)=thetat])-zs, e, N+1):

In [34]:
map(simplify,maybe);

$$\mathrm{O}\left(\varepsilon^{3}\right)$$